## Intro

This is an example notebook for demonstrating how the new [autogen>=0.4](https://github.com/microsoft/autogen) can utilize the LLMs hosted on [IBM® watsonx.ai™](https://www.ibm.com/products/watsonx-ai) by using the [autogen-watsonx-client](https://github.com/tsinggggg/autogen-watsonx-client)

### prerequisites

- pip install --upgrade autogen-watsonx-client
- pip install --upgrade autogen-agentchat>=0.4 --pre
- access to a watsonx.ai instance, setting up environment variables `WATSONX_API_KEY`, one of `WATSONX_SPACE_ID` or `WATSONX_PROJECT_ID`, optionally `WATSONX_URL`

### this is an example usage of the Swarm pattern with 2 agents

In [1]:
from typing import Any, Dict, List
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console

from autogen_watsonx_client.config import WatsonxClientConfiguration
from autogen_watsonx_client.client import WatsonXChatCompletionClient

In [2]:
wx_config = WatsonxClientConfiguration(
    model_id="meta-llama/llama-3-3-70b-instruct",  # pick a model you have access to on wx.ai here
    api_key=os.environ.get("WATSONX_API_KEY"),
    url=os.environ.get("WATSONX_URL"),
    space_id=os.environ.get("WATSONX_SPACE_ID"),
    project_id=os.environ.get("WATSONX_PROJECT_ID"),
    temperature=0.2,
)

wx_client = WatsonXChatCompletionClient(**wx_config)

### Tools

In [3]:
def refund_flight(flight_id: int) -> str:
    """Refund a flight"""
    return f"Flight {flight_id} refunded"

In [4]:
travel_agent = AssistantAgent(
    "travel_agent",
    model_client=wx_client,
    handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Say TERMINATE in natural language when the travel planning is complete.""",
)

flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=wx_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You have the ability to refund a flight using the refund_flight tool. Only make one tool call at a time.
    Do NOT make up arguments. Always ask user in natural language to provide the arguments, then handoff to user.
    Extremely important: Do NOT handoff to user without sending your message.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)

In [5]:
termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")
team = Swarm([travel_agent, flights_refunder], termination_condition=termination)

In [6]:
task = "I need to refund my flight."


async def run_team_stream() -> None:
    task_result = await Console(team.run_stream(task=task))
    last_message = task_result.messages[-1]

    while isinstance(last_message, HandoffMessage) and last_message.target == "user":
        user_message = input("User: ")

        task_result = await Console(
            team.run_stream(task=HandoffMessage(source="user", target=last_message.source, content=user_message))
        )
        last_message = task_result.messages[-1]


await run_team_stream()

---------- user ----------
I need to refund my flight.
---------- travel_agent ----------
[FunctionCall(id='chatcmpl-tool-105e2a3519f843948a0529a98ecbca61', arguments='{}', name='transfer_to_flights_refunder')]
[Prompt tokens: 269, Completion tokens: 23]
---------- travel_agent ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', call_id='chatcmpl-tool-105e2a3519f843948a0529a98ecbca61')]
---------- travel_agent ----------
Transferred to flights_refunder, adopting the role of flights_refunder immediately.
---------- flights_refunder ----------
To proceed with the refund, I will need to know the flight ID of the flight you would like to refund. Please provide the flight ID.
[Prompt tokens: 416, Completion tokens: 30]
---------- flights_refunder ----------
[FunctionCall(id='chatcmpl-tool-a4d8783e5ef14cb797e6571b6589f23f', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 450, Completion tokens: 21]
-----

User:  sure, it's 9527


---------- user ----------
sure, it's 9527
---------- flights_refunder ----------
[FunctionCall(id='chatcmpl-tool-6de6416f3f144bab805d1dba28e0dd5d', arguments='{"flight_id": "9527"}', name='refund_flight')]
[Prompt tokens: 503, Completion tokens: 26]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Flight 9527 refunded', call_id='chatcmpl-tool-6de6416f3f144bab805d1dba28e0dd5d')]
---------- flights_refunder ----------
[FunctionCall(id='chatcmpl-tool-0da53053a18f4bf6b61b3fb752ab04d5', arguments='{}', name='transfer_to_travel_agent')]
[Prompt tokens: 542, Completion tokens: 15]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Transferred to travel_agent, adopting the role of travel_agent immediately.', call_id='chatcmpl-tool-0da53053a18f4bf6b61b3fb752ab04d5')]
---------- flights_refunder ----------
Transferred to travel_agent, adopting the role of travel_agent immediately.
---------- travel_agent ----------
[FunctionCall(id='chatcmpl-tool-5c

User:  no i am good


---------- user ----------
no i am good
---------- flights_refunder ----------
You're all set then. If you have any other flights you'd like to refund in the future, don't hesitate to reach out. Have a great day!
[Prompt tokens: 683, Completion tokens: 34]
---------- flights_refunder ----------
[FunctionCall(id='chatcmpl-tool-52909d8b80ea450cabcc8877e0ccf0b8', arguments='{}', name='transfer_to_travel_agent')]
[Prompt tokens: 721, Completion tokens: 15]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Transferred to travel_agent, adopting the role of travel_agent immediately.', call_id='chatcmpl-tool-52909d8b80ea450cabcc8877e0ccf0b8')]
---------- flights_refunder ----------
Transferred to travel_agent, adopting the role of travel_agent immediately.
---------- travel_agent ----------
TERMINATE
[Prompt tokens: 562, Completion tokens: 4]
---------- Summary ----------
Number of messages: 6
Finish reason: Text 'TERMINATE' mentioned
Total prompt tokens: 1966
Total comp